In [2]:
from tqdm import tqdm
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense

import tensorflowjs as tfjs

from python.database import Database

ImportError: cannot import name 'add_to_collection'

In [2]:
db = Database()
query = 'SELECT doodle_id, tune_id FROM doodles'
lookup = dict(db.query(query))

doodles = pd.read_csv('data/doodle_images.csv').set_index(['row', 'column'])
tunes = pd.read_csv('data/music_images.csv').set_index(['row', 'column'])

In [3]:
idx = [i for i in doodles.index]
columns = []
for r, c in idx:
    for p in ['r', 'g', 'b']:
        s = 'r%i_c%i_%s' % (r, c, p)
        columns.append(s)
        
data = []
for d in tqdm(doodles.T.index):
    to_append = [d]
    pixels = doodles.T.loc[d]
    for p in pixels.tolist():
        p = p[1:-1].split(' ')
        for i in range(3):
            try:
                to_append.append(int(p[i]))
            except:
                to_append.append(0)
    data.append(to_append)

doodle_frame = pd.DataFrame(columns=['doodle_id'] + columns, data=data)
doodle_frame.set_index('doodle_id', inplace=True)

100%|██████████| 111/111 [00:00<00:00, 198.06it/s]


In [80]:
idx = [i for i in tunes.index]
columns = []
for r, c in idx:
    for p in ['r', 'g', 'b']:
        s = 'r%i_c%i_%s' % (r, c, p)
        columns.append(s)
        
data = []
for t in tqdm(tunes.T.index):
    to_append = [t]
    pixels = tunes.T.loc[t]
    for p in pixels.tolist():
        p = eval(p)
        if p == 0:
            p = (0, 0, 0)
        else:
            p = (1, 1, 1)
        for i in range(3):
            try:
                to_append.append(p[i])
            except:
                to_append.append(0)
                
    data.append(to_append)
    
tune_frame = pd.DataFrame(columns=['tune_id'] + columns, data=data)
tune_frame.set_index('tune_id', inplace=True)

 63%|██████▎   | 1270/2001 [00:13<00:07, 97.09it/s] 

KeyboardInterrupt: 

In [4]:
idx = [i for i in tunes.index]
columns = []
for r, c in idx:
    s = 'r%i_c%i' % (r, c)
    columns.append(s)
        
data = []
for t in tqdm(tunes.T.index):
    to_append = [t]
    pixels = tunes.T.loc[t]
    for p in pixels.tolist():
        p = eval(p)
        if p == 0:
            to_append.append(0)
        else:
            to_append.append(1)
                
    data.append(to_append)
    
tune_frame = pd.DataFrame(columns=['tune_id'] + columns, data=data)
tune_frame.set_index('tune_id', inplace=True)

100%|██████████| 2001/2001 [00:19<00:00, 103.07it/s]


In [5]:
X = doodle_frame.values
y = []

l = set(lookup.keys())
t = set(doodles.T.index.tolist())

diff = l.difference(t)

for d, t in lookup.items():
    if d not in diff:
        y.append(tune_frame.iloc[t].tolist())

import numpy as np
y = np.asarray(y)

print('X shape:', X.shape)
print('y shape:', y.shape)

X shape: (111, 2352)
y shape: (111, 160)


In [6]:
from sklearn.preprocessing import StandardScaler

X = StandardScaler().fit_transform(X)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [7]:
model = Sequential()
model.add(Dense(512, input_shape=(2352,), activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(160, activation='relu'))

model.compile(optimizer='adam',
              loss='mse',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               1204736   
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 160)               82080     
Total params: 1,549,472
Trainable params: 1,549,472
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.fit(X, y, batch_size=1, epochs=10)

Epoch 1/10
111/111 [==============================] - 6s 57ms/step - loss: 0.0890 - acc: 0.0270
Epoch 2/10
111/111 [==============================] - 6s 54ms/step - loss: 0.0725 - acc: 0.0090
Epoch 3/10
111/111 [==============================] - 6s 58ms/step - loss: 0.0743 - acc: 0.0000e+00
Epoch 4/10
111/111 [==============================] - 7s 60ms/step - loss: 0.0743 - acc: 0.0000e+00
Epoch 5/10
111/111 [==============================] - 6s 57ms/step - loss: 0.0713 - acc: 0.0000e+00
Epoch 6/10
111/111 [==============================] - 7s 65ms/step - loss: 0.0714 - acc: 0.0000e+00
Epoch 7/10
111/111 [==============================] - 7s 61ms/step - loss: 0.0713 - acc: 0.0000e+00
Epoch 8/10
111/111 [==============================] - 7s 60ms/step - loss: 0.0713 - acc: 0.0000e+00
Epoch 9/10
111/111 [==============================] - 6s 58ms/step - loss: 0.0713 - acc: 0.0000e+00
Epoch 10/10
111/111 [==============================] - 6s 56ms/step - loss: 0.0713 - acc: 0.0000e+00


In [13]:
evaluate = np.vectorize(lambda x: 1 if x > 0 else 0)

In [14]:
x = model.predict(X[2].reshape(1, X.shape[1]))
evaluate(x[0])